# Caso Práctico Unidad 3 · Generative AI
## Aplicación web de generación de imágenes y edición de contenido con Amazon Bedrock (Stable Diffusion + Claude)

Este notebook implementa una **demo funcional** (tipo *web app*) con **Gradio** que cubre los requisitos del enunciado:
- Generación de imágenes con **Stable Diffusion XL** en **Amazon Bedrock** (estilos, parámetros, galería y descarga).
- Edición de contenido con **Claude** en **Amazon Bedrock** (resumir, expandir, corregir, variaciones).
- Historial de versiones (comparar y revertir), comentarios y colaboración.
- Roles y permisos (RBAC).
- Medidas de seguridad: cifrado en reposo (demo con Fernet) y moderación/filtro de contenido (Guardrails opcional + fallback).


## 0) Requisitos del enunciado (resumen)
La aplicación debe incluir:
1. **Generación de imágenes** (prompt + estilos + galería/descarga).
2. **Edición de contenido** con Claude (resumen/expansión/corrección/variaciones) y **versionado** con revert.
3. **Colaboración y flujo de trabajo**: multiusuario, roles/permisos, comentarios/notas.
4. **Consideraciones éticas y seguridad**: privacidad/cifrado, pautas éticas, moderación para evitar contenido inapropiado.

*(Ver enunciado Unidad 3, páginas 2-3.)*


## 1) Instalación de dependencias (opcional)


In [1]:
# Si estás en un entorno gestionado que ya trae dependencias, puedes omitir.
!pip -q install boto3 botocore pillow gradio pandas python-dotenv cryptography


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.9 MB/s eta 0:00:00


## 2) Imports y configuración


In [2]:
import os, json, base64, uuid, time, random, sqlite3, pathlib, tempfile
from datetime import datetime
from typing import Optional, Dict, Any, List, Tuple

import boto3
from botocore.exceptions import ClientError, NoCredentialsError

from PIL import Image
from io import BytesIO

from cryptography.fernet import Fernet, InvalidToken
import gradio as gr


In [3]:
# =========================
# Configuración (ENV)
# =========================
REGION = os.getenv("AWS_REGION", "eu-west-1")

# Model IDs (ajusta según región y modelos habilitados)
MODEL_ID_IMAGE = os.getenv("BEDROCK_SD_MODEL_ID", "stability.stable-diffusion-xl-v1")
MODEL_ID_TEXT  = os.getenv("BEDROCK_CLAUDE_MODEL_ID", "anthropic.claude-3-5-sonnet-20240620-v1:0")

# Directorios y BD
OUTPUT_DIR = pathlib.Path(os.getenv("OUTPUT_DIR", "output"))
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DB_PATH = os.getenv("DB_PATH", "unit3_genai_app.db")

# Guardrails (opcional). Si no se define, se usa fallback local.
GUARDRAIL_ID = os.getenv("BEDROCK_GUARDRAIL_ID", "").strip()
GUARDRAIL_VERSION = os.getenv("BEDROCK_GUARDRAIL_VERSION", "DRAFT").strip()  # DRAFT o 1,2,...

# Modo demo (si no hay credenciales o para pruebas locales)
MOCK_MODE = os.getenv("MOCK_MODE", "0").strip() == "1"

print("REGION:", REGION)
print("MODEL_ID_IMAGE:", MODEL_ID_IMAGE)
print("MODEL_ID_TEXT :", MODEL_ID_TEXT)
print("OUTPUT_DIR    :", OUTPUT_DIR.resolve())
print("DB_PATH       :", pathlib.Path(DB_PATH).resolve())
print("GUARDRAIL_ID  :", GUARDRAIL_ID or "(no configurado)")
print("MOCK_MODE     :", MOCK_MODE)


REGION: eu-west-1
MODEL_ID_IMAGE: stability.stable-diffusion-xl-v1
MODEL_ID_TEXT : anthropic.claude-3-5-sonnet-20240620-v1:0
OUTPUT_DIR    : /content/output
DB_PATH       : /content/unit3_genai_app.db
GUARDRAIL_ID  : (no configurado)
MOCK_MODE     : False


## 3) Cliente Bedrock Runtime (Boto3)


In [4]:
def get_bedrock_runtime(region: str = REGION):
    return boto3.client("bedrock-runtime", region_name=region)

def bedrock_healthcheck() -> Tuple[bool, str]:
    """Comprueba si podemos crear cliente y tenemos credenciales resolubles."""
    if MOCK_MODE:
        return True, "MOCK_MODE=1 (no se llamará a AWS)."
    try:
        _ = get_bedrock_runtime(REGION)
        # Forzamos resolución de credenciales intentando firmar una llamada “inocua”.
        # No hay endpoint ping, así que simplemente verificamos que boto3 encuentre credenciales.
        session = boto3.Session()
        creds = session.get_credentials()
        if creds is None:
            return False, "No se encontraron credenciales AWS (boto3 no resolvió credenciales)."
        return True, "Cliente Bedrock listo y credenciales detectadas."
    except NoCredentialsError:
        return False, "NoCredentialsError: credenciales no configuradas."
    except Exception as e:
        return False, f"Error inicializando Bedrock: {e}"

ok, msg = bedrock_healthcheck()
print(msg)


No se encontraron credenciales AWS (boto3 no resolvió credenciales).


## 4) Base de datos SQLite (usuarios, imágenes, documentos, versiones, comentarios, aprobaciones)


In [5]:
def db_connect(db_path: str = DB_PATH):
    con = sqlite3.connect(db_path, check_same_thread=False)
    con.row_factory = sqlite3.Row
    return con

def db_init():
    con = db_connect()
    cur = con.cursor()

    # Usuarios y roles
    cur.execute("""CREATE TABLE IF NOT EXISTS users(
        user_id TEXT PRIMARY KEY,
        username TEXT NOT NULL UNIQUE,
        role TEXT NOT NULL,
        created_at TEXT NOT NULL
    )""")

    # Imágenes (galería)
    cur.execute("""CREATE TABLE IF NOT EXISTS images(
        image_id TEXT PRIMARY KEY,
        user_id TEXT NOT NULL,
        prompt TEXT NOT NULL,
        style TEXT NOT NULL,
        seed INTEGER,
        steps INTEGER,
        cfg_scale REAL,
        file_path TEXT NOT NULL,
        encrypted INTEGER NOT NULL DEFAULT 0,
        created_at TEXT NOT NULL
    )""")

    # Documentos y versiones
    cur.execute("""CREATE TABLE IF NOT EXISTS documents(
        doc_id TEXT PRIMARY KEY,
        title TEXT NOT NULL,
        created_by TEXT NOT NULL,
        created_at TEXT NOT NULL
    )""")

    cur.execute("""CREATE TABLE IF NOT EXISTS versions(
        version_id TEXT PRIMARY KEY,
        doc_id TEXT NOT NULL,
        user_id TEXT NOT NULL,
        operation TEXT NOT NULL,
        content TEXT NOT NULL,
        encrypted INTEGER NOT NULL DEFAULT 0,
        created_at TEXT NOT NULL
    )""")

    # Comentarios / notas
    cur.execute("""CREATE TABLE IF NOT EXISTS comments(
        comment_id TEXT PRIMARY KEY,
        doc_id TEXT NOT NULL,
        version_id TEXT,
        user_id TEXT NOT NULL,
        comment TEXT NOT NULL,
        created_at TEXT NOT NULL
    )""")

    # Aprobaciones (flujo de trabajo básico)
    cur.execute("""CREATE TABLE IF NOT EXISTS approvals(
        approval_id TEXT PRIMARY KEY,
        doc_id TEXT NOT NULL,
        version_id TEXT NOT NULL,
        approver_id TEXT NOT NULL,
        status TEXT NOT NULL, -- pending/approved/rejected
        note TEXT,
        created_at TEXT NOT NULL
    )""")

    con.commit()
    con.close()

db_init()
print("SQLite inicializado.")


SQLite inicializado.


## 5) Cifrado en reposo (demo)


In [6]:
# En producción: S3 + KMS, Secrets Manager, rotación de claves, etc.
# En esta demo: Fernet (clave simétrica) en variable de entorno ENCRYPTION_KEY.

def get_fernet() -> Optional[Fernet]:
    key = os.getenv("ENCRYPTION_KEY", "").strip()
    if not key:
        # Generamos una clave sólo para la sesión (para demo). Recomienda persistir en ENV.
        key = Fernet.generate_key().decode("utf-8")
        os.environ["ENCRYPTION_KEY"] = key
        print("⚠️ ENCRYPTION_KEY no estaba definida. Se generó una temporal para esta sesión.")
    try:
        return Fernet(key.encode("utf-8"))
    except Exception as e:
        print(f"⚠️ ENCRYPTION_KEY inválida: {e}")
        return None

FERNET = get_fernet()

def encrypt_text(plain: str) -> Tuple[str, int]:
    if not FERNET:
        return plain, 0
    token = FERNET.encrypt(plain.encode("utf-8")).decode("utf-8")
    return token, 1

def decrypt_text(cipher: str, encrypted_flag: int) -> str:
    if encrypted_flag != 1 or not FERNET:
        return cipher
    try:
        return FERNET.decrypt(cipher.encode("utf-8")).decode("utf-8")
    except InvalidToken:
        # Clave distinta a la usada para cifrar
        return "[ERROR] No se pudo descifrar (clave incorrecta)."

def encrypt_file_bytes(data: bytes) -> Tuple[bytes, int]:
    if not FERNET:
        return data, 0
    return FERNET.encrypt(data), 1

def decrypt_file_bytes(data: bytes, encrypted_flag: int) -> bytes:
    if encrypted_flag != 1 or not FERNET:
        return data
    return FERNET.decrypt(data)


⚠️ ENCRYPTION_KEY no estaba definida. Se generó una temporal para esta sesión.


## 6) Moderación / filtrado de contenido


In [7]:
# 6.1 Guardrails (opcional) + fallback local
# - Si GUARDRAIL_ID está configurado, se evalúa INPUT/OUTPUT.
# - Si no, se aplica un filtro mínimo por palabras clave (demo).

BASIC_BLOCKLIST = {
    # Demo (lista mínima, ajusta a tus políticas)
    "porn", "child", "rape", "terrorist", "bomb", "nazis", "suicide"
}

def apply_guardrail_text(
    text: str,
    source: str = "INPUT",  # INPUT | OUTPUT
    region: str = REGION,
) -> Dict[str, Any]:
    if not GUARDRAIL_ID:
        return {"action": "NONE", "outputs": [], "note": "Guardrail no configurado"}
    rt = get_bedrock_runtime(region)
    resp = rt.apply_guardrail(
        guardrailIdentifier=GUARDRAIL_ID,
        guardrailVersion=GUARDRAIL_VERSION,
        source=source,
        content=[{"text": {"text": text}}],
    )
    return resp

def basic_filter(text: str) -> Tuple[bool, str]:
    t = (text or "").lower()
    for w in BASIC_BLOCKLIST:
        if w in t:
            return False, f"Bloqueado por política (keyword: {w})"
    return True, "OK"

def moderate_text(text: str, source: str="INPUT") -> Tuple[bool, str]:
    # 1) Guardrails si existe
    if GUARDRAIL_ID and not MOCK_MODE:
        try:
            verdict = apply_guardrail_text(text=text, source=source)
            action = verdict.get("action", "NONE")
            if action and action.upper() in {"BLOCKED", "BLOCK"}:
                return False, f"Bloqueado por Guardrails ({action})."
            return True, "OK (Guardrails)"
        except Exception as e:
            # fallback si guardrails falla
            pass
    # 2) fallback local
    return basic_filter(text)


## 7) Roles y permisos (RBAC)


In [8]:
ROLE_PERMISSIONS = {
    "designer":  {"generate_image", "view_gallery", "comment", "view_versions"},
    "copywriter":{"edit_text", "create_doc", "view_versions", "revert_version", "comment"},
    "approver":  {"view_gallery", "view_versions", "comment", "approve"},
    "admin":     {"generate_image", "view_gallery", "edit_text", "create_doc", "view_versions",
                  "revert_version", "comment", "approve", "manage_users"},
}

def has_permission(role: str, action: str) -> bool:
    return action in ROLE_PERMISSIONS.get(role, set())

def ensure_permission(role: str, action: str):
    if not has_permission(role, action):
        raise PermissionError(f"Acción '{action}' no permitida para rol '{role}'")

def seed_demo_users():
    con = db_connect()
    cur = con.cursor()
    now = datetime.utcnow().isoformat()

    demo = [
        ("u1", "ana_disenio", "designer"),
        ("u2", "carlos_copy", "copywriter"),
        ("u3", "marta_aprueba", "approver"),
        ("u4", "admin", "admin"),
    ]
    for uid, uname, role in demo:
        cur.execute("""INSERT OR IGNORE INTO users(user_id, username, role, created_at)
                       VALUES(?,?,?,?)""", (uid, uname, role, now))
    con.commit()
    con.close()

def get_users() -> List[Tuple[str,str,str]]:
    con = db_connect()
    cur = con.cursor()
    rows = cur.execute("SELECT user_id, username, role FROM users ORDER BY username").fetchall()
    con.close()
    return [(r["user_id"], r["username"], r["role"]) for r in rows]

seed_demo_users()
print("Usuarios demo creados (si no existían).")
print(get_users())


Usuarios demo creados (si no existían).
[('u4', 'admin', 'admin'), ('u1', 'ana_disenio', 'designer'), ('u2', 'carlos_copy', 'copywriter'), ('u3', 'marta_aprueba', 'approver')]


/tmp/ipython-input-962457123.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow().isoformat()


## 8) Amazon Bedrock · Stable Diffusion (generación de imágenes)


In [9]:
STYLE_PRESETS = {
    "Realismo (photographic)": "photographic",
    "Anime": "anime",
    "Digital Art": "digital-art",
    "Cinematic": "cinematic",
    "Oil Painting": "oil-painting",
    "3D Model": "3d-model",
    "Pixel Art": "pixel-art",
}

def invoke_stable_diffusion_xl(
    prompt: str,
    style_preset: str = "photographic",
    negative_prompt: Optional[str] = None,
    seed: Optional[int] = None,
    steps: int = 30,
    cfg_scale: float = 7.0,
    width: int = 1024,
    height: int = 1024,
    model_id: str = MODEL_ID_IMAGE,
    region: str = REGION,
) -> Tuple[Image.Image, Dict[str, Any]]:
    """Invoca Stable Diffusion XL en Bedrock y devuelve PIL.Image + metadata."""
    # Moderación del prompt (INPUT)
    ok, reason = moderate_text(prompt, source="INPUT")
    if not ok:
        raise ValueError(f"Prompt rechazado: {reason}")

    if MOCK_MODE:
        # Imagen dummy para pruebas locales
        img = Image.new("RGB", (width, height), color=(30, 30, 40))
        meta = {"model_id": "MOCK", "seed": seed or 0, "steps": steps, "cfg_scale": cfg_scale, "style_preset": style_preset,
                "width": width, "height": height}
        return img, meta

    rt = get_bedrock_runtime(region)

    if seed is None or seed == -1:
        seed = random.randint(0, 2**32 - 1)

    payload: Dict[str, Any] = {
        "text_prompts": [{"text": prompt, "weight": 1.0}],
        "cfg_scale": float(cfg_scale),
        "seed": int(seed),
        "steps": int(steps),
        "style_preset": style_preset,
        "width": int(width),
        "height": int(height),
    }

    if negative_prompt:
        # Algunas variantes aceptan negative prompt como prompt adicional con weight negativa.
        payload["text_prompts"].append({"text": negative_prompt, "weight": -1.0})

    try:
        resp = rt.invoke_model(
            modelId=model_id,
            contentType="application/json",
            accept="application/json",
            body=json.dumps(payload),
        )
        body = json.loads(resp["body"].read())

        # Estructura típica (Stability): {'artifacts': [{'base64': '...', 'seed': ...}], ...}
        artifacts = body.get("artifacts", [])
        if not artifacts:
            raise RuntimeError(f"Respuesta inesperada de SD: keys={list(body.keys())}")

        b64 = artifacts[0].get("base64")
        if not b64:
            raise RuntimeError("No se encontró 'base64' en artifacts[0].")

        img_bytes = base64.b64decode(b64)
        img = Image.open(BytesIO(img_bytes)).convert("RGB")

        meta = {
            "model_id": model_id,
            "seed": artifacts[0].get("seed", seed),
            "steps": steps,
            "cfg_scale": cfg_scale,
            "style_preset": style_preset,
            "width": width,
            "height": height,
            "raw_response_keys": list(body.keys()),
        }
        return img, meta

    except ClientError as e:
        raise RuntimeError(f"Error invocando Stable Diffusion: {e}") from e


### 8.1 Guardar imagen (cifrada opcional) + registrar en BD


In [10]:
def save_image_and_register(
    img: Image.Image,
    user_id: str,
    prompt: str,
    style_label: str,
    meta: Dict[str, Any],
    encrypt_at_rest: bool = True,
) -> str:
    image_id = str(uuid.uuid4())
    ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    filename = f"{ts}_{image_id}.png"
    out_path = OUTPUT_DIR / filename

    # Guardado (bytes)
    buf = BytesIO()
    img.save(buf, format="PNG")
    raw = buf.getvalue()

    data, enc_flag = (encrypt_file_bytes(raw) if encrypt_at_rest else (raw, 0))
    out_path.write_bytes(data)

    con = db_connect()
    cur = con.cursor()
    cur.execute("""INSERT INTO images(image_id, user_id, prompt, style, seed, steps, cfg_scale, file_path, encrypted, created_at)
                   VALUES(?,?,?,?,?,?,?,?,?,?)""",
                (
                    image_id,
                    user_id,
                    prompt,
                    style_label,
                    int(meta.get("seed")) if meta.get("seed") is not None else None,
                    int(meta.get("steps")) if meta.get("steps") is not None else None,
                    float(meta.get("cfg_scale")) if meta.get("cfg_scale") is not None else None,
                    str(out_path),
                    int(enc_flag),
                    datetime.utcnow().isoformat(),
                ))
    con.commit()
    con.close()

    return str(out_path)

def list_gallery(limit: int = 20) -> List[Dict[str, Any]]:
    con = db_connect()
    cur = con.cursor()
    rows = cur.execute("""SELECT * FROM images ORDER BY created_at DESC LIMIT ?""", (limit,)).fetchall()
    con.close()
    return [dict(r) for r in rows]

def load_image_for_display(file_path: str, encrypted_flag: int) -> Image.Image:
    data = pathlib.Path(file_path).read_bytes()
    data = decrypt_file_bytes(data, encrypted_flag)
    img = Image.open(BytesIO(data)).convert("RGB")
    return img

def export_image_tempfile(file_path: str, encrypted_flag: int) -> str:
    """Devuelve una ruta temporal con PNG descifrado para descarga."""
    data = pathlib.Path(file_path).read_bytes()
    data = decrypt_file_bytes(data, encrypted_flag)
    fd, tmp_path = tempfile.mkstemp(suffix=".png")
    os.close(fd)
    pathlib.Path(tmp_path).write_bytes(data)
    return tmp_path


## 9) Amazon Bedrock · Claude (edición de contenido)


In [11]:
def invoke_claude_messages(
    user_text: str,
    system_text: Optional[str] = None,
    max_tokens: int = 800,
    temperature: float = 0.4,
    model_id: str = MODEL_ID_TEXT,
    region: str = REGION,
) -> Dict[str, Any]:
    """Invoca Claude (messages API) en Bedrock y devuelve el JSON de respuesta."""
    ok_in, reason_in = moderate_text(user_text, source="INPUT")
    if not ok_in:
        raise ValueError(f"Texto de entrada rechazado: {reason_in}")

    if MOCK_MODE:
        # Respuesta dummy para pruebas locales
        return {"content": [{"type": "text", "text": f"[MOCK] Procesado: {user_text[:200]}"}]}

    rt = get_bedrock_runtime(region)

    messages = [{
        "role": "user",
        "content": [{"type": "text", "text": user_text}]
    }]

    payload: Dict[str, Any] = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": int(max_tokens),
        "temperature": float(temperature),
        "messages": messages,
    }

    if system_text:
        payload["system"] = system_text

    try:
        resp = rt.invoke_model(
            modelId=model_id,
            contentType="application/json",
            accept="application/json",
            body=json.dumps(payload),
        )
        body = json.loads(resp["body"].read())

        # Moderación del output (OUTPUT) si hay guardrails
        out_text = claude_text_only(body)
        ok_out, reason_out = moderate_text(out_text, source="OUTPUT")
        if not ok_out:
            raise ValueError(f"Salida rechazada por política: {reason_out}")

        return body
    except ClientError as e:
        raise RuntimeError(f"Error invocando Claude: {e}") from e

def claude_text_only(response_json: Dict[str, Any]) -> str:
    content = response_json.get("content", [])
    texts = []
    for c in content:
        if c.get("type") == "text":
            texts.append(c.get("text", ""))
    return "\n".join(texts).strip()


### 9.1 Operaciones de edición


In [12]:
EDIT_PROMPTS = {
    "Resumir": {
        "system": "Eres un asistente de redacción. Resume de forma clara y profesional.",
        "template": "Resume el siguiente texto en 5-7 líneas, manteniendo los puntos clave:\n\n{input}",
    },
    "Expandir ideas": {
        "system": "Eres un asistente de redacción. Expande ideas con claridad, sin inventar datos.",
        "template": "Expande el siguiente texto añadiendo detalle, estructura y ejemplos. No inventes hechos:\n\n{input}",
    },
    "Corregir gramática/estilo": {
        "system": "Eres un corrector profesional. Corrige gramática y estilo manteniendo el significado.",
        "template": "Corrige el siguiente texto (gramática, ortografía y estilo) manteniendo el significado:\n\n{input}",
    },
    "Generar 3 variaciones": {
        "system": "Eres un copywriter. Genera variaciones manteniendo el mensaje y el tono.",
        "template": "Genera 3 variaciones del siguiente texto (estilo profesional). Separa cada variación con '---':\n\n{input}",
    },
}

def edit_text_with_claude(operation: str, input_text: str) -> str:
    if operation not in EDIT_PROMPTS:
        raise ValueError(f"Operación no soportada: {operation}")
    cfg = EDIT_PROMPTS[operation]
    prompt = cfg["template"].format(input=input_text)
    resp = invoke_claude_messages(user_text=prompt, system_text=cfg["system"])
    return claude_text_only(resp)


## 10) Documentos, versionado, diff y revert


In [13]:
import difflib

def create_document(title: str, created_by: str, initial_content: str, encrypt_at_rest: bool=True) -> str:
    doc_id = str(uuid.uuid4())
    now = datetime.utcnow().isoformat()

    con = db_connect()
    cur = con.cursor()
    cur.execute("""INSERT INTO documents(doc_id, title, created_by, created_at) VALUES(?,?,?,?)""",
                (doc_id, title, created_by, now))
    con.commit()
    con.close()

    # Primera versión
    add_doc_version(doc_id, created_by, "create", initial_content, encrypt_at_rest=encrypt_at_rest)
    return doc_id

def list_documents(limit: int=50) -> List[Dict[str,Any]]:
    con = db_connect()
    cur = con.cursor()
    rows = cur.execute("""SELECT * FROM documents ORDER BY created_at DESC LIMIT ?""", (limit,)).fetchall()
    con.close()
    return [dict(r) for r in rows]

def add_doc_version(doc_id: str, user_id: str, operation: str, content: str, encrypt_at_rest: bool=True) -> str:
    version_id = str(uuid.uuid4())
    now = datetime.utcnow().isoformat()

    content_store, enc_flag = (encrypt_text(content) if encrypt_at_rest else (content, 0))

    con = db_connect()
    cur = con.cursor()
    cur.execute("""INSERT INTO versions(version_id, doc_id, user_id, operation, content, encrypted, created_at)
                   VALUES(?,?,?,?,?,?,?)""",
                (version_id, doc_id, user_id, operation, content_store, int(enc_flag), now))
    con.commit()
    con.close()
    return version_id

def get_doc_versions(doc_id: str) -> List[Dict[str,Any]]:
    con = db_connect()
    cur = con.cursor()
    rows = cur.execute("""SELECT version_id, doc_id, user_id, operation, content, encrypted, created_at
                          FROM versions WHERE doc_id = ? ORDER BY created_at DESC""", (doc_id,)).fetchall()
    con.close()
    out=[]
    for r in rows:
        d=dict(r)
        d["content_plain"]=decrypt_text(d["content"], d["encrypted"])
        out.append(d)
    return out

def get_version_content(version_id: str) -> str:
    con = db_connect()
    cur = con.cursor()
    row = cur.execute("""SELECT content, encrypted FROM versions WHERE version_id = ?""", (version_id,)).fetchone()
    con.close()
    if not row:
        raise ValueError("Versión no encontrada.")
    return decrypt_text(row["content"], row["encrypted"])

def diff_versions(old_text: str, new_text: str) -> str:
    diff = difflib.unified_diff(
        old_text.splitlines(),
        new_text.splitlines(),
        fromfile="old",
        tofile="new",
        lineterm=""
    )
    return "\n".join(diff)

def revert_to_version(doc_id: str, user_id: str, target_version_id: str) -> str:
    content = get_version_content(target_version_id)
    return add_doc_version(doc_id, user_id, "revert", content, encrypt_at_rest=True)


## 11) Comentarios y aprobaciones


In [14]:
def add_comment(doc_id: str, user_id: str, comment: str, version_id: Optional[str]=None) -> str:
    comment_id = str(uuid.uuid4())
    now = datetime.utcnow().isoformat()
    con = db_connect()
    cur = con.cursor()
    cur.execute("""INSERT INTO comments(comment_id, doc_id, version_id, user_id, comment, created_at)
                   VALUES(?,?,?,?,?,?)""", (comment_id, doc_id, version_id, user_id, comment, now))
    con.commit()
    con.close()
    return comment_id

def list_comments(doc_id: str, limit: int=100) -> List[Dict[str,Any]]:
    con = db_connect()
    cur = con.cursor()
    rows = cur.execute("""SELECT * FROM comments WHERE doc_id = ? ORDER BY created_at DESC LIMIT ?""", (doc_id, limit)).fetchall()
    con.close()
    return [dict(r) for r in rows]

def set_approval(doc_id: str, version_id: str, approver_id: str, status: str, note: str="") -> str:
    if status not in {"pending","approved","rejected"}:
        raise ValueError("status inválido")
    approval_id = str(uuid.uuid4())
    now = datetime.utcnow().isoformat()
    con = db_connect()
    cur = con.cursor()
    cur.execute("""INSERT INTO approvals(approval_id, doc_id, version_id, approver_id, status, note, created_at)
                   VALUES(?,?,?,?,?,?,?)""", (approval_id, doc_id, version_id, approver_id, status, note, now))
    con.commit()
    con.close()
    return approval_id

def list_approvals(doc_id: str, limit: int=50) -> List[Dict[str,Any]]:
    con = db_connect()
    cur = con.cursor()
    rows = cur.execute("""SELECT * FROM approvals WHERE doc_id = ? ORDER BY created_at DESC LIMIT ?""", (doc_id, limit)).fetchall()
    con.close()
    return [dict(r) for r in rows]


## 12) Interfaz web (Gradio)


In [17]:
def format_user_choices() -> List[str]:
    return [f"{uid} | {uname} ({role})" for uid, uname, role in get_users()]

def parse_user_choice(choice: str) -> Tuple[str, str, str]:
    # "u1 | ana_disenio (designer)"
    uid = choice.split("|", 1)[0].strip()
    uname = choice.split("|", 1)[1].split("(")[0].strip()
    role = choice.split("(")[-1].split(")")[0].strip()
    return uid, uname, role

def ui_generate_image(user_choice, prompt, style_label, negative_prompt, steps, cfg_scale, seed, width, height, encrypt_store):
    user_id, uname, role = parse_user_choice(user_choice)
    ensure_permission(role, "generate_image")

    style_preset = STYLE_PRESETS.get(style_label, "photographic")
    seed_val = None if seed in (None, "", -1) else int(seed)

    img, meta = invoke_stable_diffusion_xl(
        prompt=prompt,
        style_preset=style_preset,
        negative_prompt=negative_prompt or None,
        seed=seed_val,
        steps=int(steps),
        cfg_scale=float(cfg_scale),
        width=int(width),
        height=int(height),
    )
    saved_path = save_image_and_register(img, user_id, prompt, style_label, meta, encrypt_at_rest=bool(encrypt_store))
    return img, json.dumps(meta, indent=2), saved_path

def ui_refresh_gallery(limit):
    rows = list_gallery(int(limit))
    # devolvemos lista de imágenes (PIL) y tabla de metadatos
    imgs=[]
    meta=[]
    for r in rows:
        try:
            imgs.append(load_image_for_display(r["file_path"], r["encrypted"]))
        except Exception:
            imgs.append(None)
        meta.append({
            "image_id": r["image_id"],
            "user_id": r["user_id"],
            "style": r["style"],
            "seed": r["seed"],
            "steps": r["steps"],
            "cfg_scale": r["cfg_scale"],
            "created_at": r["created_at"],
            "encrypted": r["encrypted"],
            "file_path": r["file_path"],
        })
    return imgs, meta

def ui_download_selected(image_row_json: str) -> str:
    r = json.loads(image_row_json)
    return export_image_tempfile(r["file_path"], int(r.get("encrypted", 0)))

def ui_create_doc(user_choice, title, initial_text):
    user_id, uname, role = parse_user_choice(user_choice)
    ensure_permission(role, "create_doc")
    ok, reason = moderate_text(initial_text, source="INPUT")
    if not ok:
        raise ValueError(f"Texto rechazado: {reason}")
    doc_id = create_document(title, user_id, initial_text, encrypt_at_rest=True)
    return doc_id, f"Documento creado: {doc_id}"

def ui_list_docs():
    docs = list_documents(50)
    # opciones para dropdown
    return [f'{d["doc_id"]} | {d["title"]} ({d["created_at"]})' for d in docs]

def ui_load_versions(doc_choice):
    if not doc_choice:
        return [], ""
    doc_id = doc_choice.split("|",1)[0].strip()
    versions = get_doc_versions(doc_id)
    options = [f'{v["version_id"]} | {v["operation"]} by {v["user_id"]} ({v["created_at"]})' for v in versions]
    latest_text = versions[0]["content_plain"] if versions else ""
    return options, latest_text

def ui_edit_text(user_choice, operation, input_text, doc_choice):
    user_id, uname, role = parse_user_choice(user_choice)
    ensure_permission(role, "edit_text")
    if not doc_choice:
        raise ValueError("Selecciona un documento.")
    doc_id = doc_choice.split("|",1)[0].strip()

    out = edit_text_with_claude(operation, input_text)
    vid = add_doc_version(doc_id, user_id, f"edit:{operation}", out, encrypt_at_rest=True)
    return out, f"Nueva versión guardada: {vid}"

def ui_diff_two_versions(doc_choice, v_old, v_new):
    if not (doc_choice and v_old and v_new):
        return ""
    old_id = v_old.split("|",1)[0].strip()
    new_id = v_new.split("|",1)[0].strip()
    return diff_versions(get_version_content(old_id), get_version_content(new_id))

def ui_revert(user_choice, doc_choice, target_version):
    user_id, uname, role = parse_user_choice(user_choice)
    ensure_permission(role, "revert_version")
    doc_id = doc_choice.split("|",1)[0].strip()
    target_id = target_version.split("|",1)[0].strip()
    new_vid = revert_to_version(doc_id, user_id, target_id)
    return f"Revert realizado. Nueva versión: {new_vid}"

def ui_add_comment(user_choice, doc_choice, version_choice, comment_text):
    user_id, uname, role = parse_user_choice(user_choice)
    ensure_permission(role, "comment")
    if not doc_choice:
        raise ValueError("Selecciona un documento.")
    doc_id = doc_choice.split("|",1)[0].strip()
    version_id = version_choice.split("|",1)[0].strip() if version_choice else None
    add_comment(doc_id, user_id, comment_text, version_id=version_id)
    return "Comentario añadido."

def ui_list_comments(doc_choice):
    if not doc_choice:
        return []
    doc_id = doc_choice.split("|",1)[0].strip()
    return list_comments(doc_id)

def ui_approve(user_choice, doc_choice, version_choice, status, note):
    user_id, uname, role = parse_user_choice(user_choice)
    ensure_permission(role, "approve")
    if not (doc_choice and version_choice):
        raise ValueError("Selecciona documento y versión.")
    doc_id = doc_choice.split("|",1)[0].strip()
    vid = version_choice.split("|",1)[0].strip()
    set_approval(doc_id, vid, user_id, status, note)
    return "Aprobación registrada."

def ui_list_approvals(doc_choice):
    if not doc_choice:
        return []
    doc_id = doc_choice.split("|",1)[0].strip()
    return list_approvals(doc_id)

# Admin - gestión simple de usuarios
def ui_add_user(admin_choice, new_username, new_role):
    admin_id, _, admin_role = parse_user_choice(admin_choice)
    ensure_permission(admin_role, "manage_users")
    uid = "u_" + str(uuid.uuid4())[:8]
    now = datetime.utcnow().isoformat()
    con = db_connect()
    cur = con.cursor()
    cur.execute("""INSERT INTO users(user_id, username, role, created_at) VALUES(?,?,?,?)""",
                (uid, new_username, new_role, now))
    con.commit()
    con.close()
    return f"Usuario creado: {uid}"

with gr.Blocks(title="Unidad 3 · Bedrock GenAI App", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""# Unidad 3 · Bedrock GenAI App (Demo)
**Multiusuario**, **roles**, **galería**, **edición de texto**, **versionado**, **comentarios** y **aprobación**.
""")

    user_choice = gr.Dropdown(label="Usuario (simula login)", choices=format_user_choices(), value=format_user_choices()[0])

    with gr.Tab("🖼️ Generar Imagen"):
        prompt = gr.Textbox(label="Prompt", lines=3, placeholder="Ej: Un paisaje nocturno con luces de neón, estilo cinematic.")
        style = gr.Dropdown(label="Estilo", choices=list(STYLE_PRESETS.keys()), value="Realismo (photographic)")
        negative_prompt = gr.Textbox(label="Negative prompt (opcional)", lines=2, placeholder="Ej: blur, low quality, watermark...")
        with gr.Row():
            steps = gr.Slider(label="Steps", minimum=10, maximum=80, value=30, step=1)
            cfg_scale = gr.Slider(label="CFG Scale", minimum=1, maximum=20, value=7, step=0.5)
        with gr.Row():
            seed = gr.Number(label="Seed (-1 aleatorio)", value=-1, precision=0)
            encrypt_store = gr.Checkbox(label="Cifrar imagen en reposo (demo)", value=True)
        with gr.Row():
            width = gr.Dropdown(label="Width", choices=[512, 768, 1024], value=1024)
            height = gr.Dropdown(label="Height", choices=[512, 768, 1024], value=1024)

        gen_btn = gr.Button("Generar")
        out_img = gr.Image(label="Imagen generada", type="pil")
        out_meta = gr.Code(label="Metadata", language="json")
        out_path = gr.Textbox(label="Ruta guardada (local)", interactive=False)

        gen_btn.click(
            fn=ui_generate_image,
            inputs=[user_choice, prompt, style, negative_prompt, steps, cfg_scale, seed, width, height, encrypt_store],
            outputs=[out_img, out_meta, out_path],
        )

    with gr.Tab("🗂️ Galería"):
        limit = gr.Slider(label="Últimas N imágenes", minimum=1, maximum=50, value=12, step=1)
        refresh_btn = gr.Button("Actualizar galería")
        gallery = gr.Gallery(label="Imágenes", columns=3, height=420)
        gallery_meta = gr.JSON(label="Metadatos (lista)")
        refresh_btn.click(fn=ui_refresh_gallery, inputs=[limit], outputs=[gallery, gallery_meta])

        gr.Markdown("""**Descarga**: copia un elemento JSON de la lista (una fila) y pégalo aquí para descargarlo.""")
        image_row_json = gr.Textbox(label="Fila JSON de la imagen (pega aquí)", lines=3)
        download_btn = gr.Button("Preparar archivo para descarga")
        download_file = gr.File(label="Descargar")
        download_btn.click(fn=ui_download_selected, inputs=[image_row_json], outputs=[download_file])

    with gr.Tab("📝 Documentos"):
        with gr.Row():
            doc_title = gr.Textbox(label="Título", placeholder="Campaña Navidad 2025")
            doc_text = gr.Textbox(label="Texto inicial", lines=5, placeholder="Pega aquí el texto base...")
        create_btn = gr.Button("Crear documento")
        created_doc_id = gr.Textbox(label="doc_id", interactive=False)
        create_msg = gr.Textbox(label="Estado", interactive=False)

        create_btn.click(fn=ui_create_doc, inputs=[user_choice, doc_title, doc_text], outputs=[created_doc_id, create_msg])

        gr.Markdown("### Selección de documento y versiones")
        reload_docs_btn = gr.Button("Refrescar lista de documentos")
        doc_choice = gr.Dropdown(label="Documento", choices=[], value=None)
        reload_docs_btn.click(fn=ui_list_docs, inputs=None, outputs=doc_choice)

        versions_dd = gr.Dropdown(label="Versiones (última primero)", choices=[], value=None)
        latest_text = gr.Textbox(label="Contenido (última versión)", lines=10)

        doc_choice.change(fn=ui_load_versions, inputs=[doc_choice], outputs=[versions_dd, latest_text])

    with gr.Tab("✍️ Editar con Claude"):
        operation = gr.Radio(label="Operación", choices=list(EDIT_PROMPTS.keys()), value="Resumir")
        input_text = gr.Textbox(label="Texto a editar", lines=8, placeholder="Texto sobre el que aplicar la operación...")
        edit_btn = gr.Button("Ejecutar edición (Claude) + guardar versión")
        edited_out = gr.Textbox(label="Salida", lines=10)
        edit_status = gr.Textbox(label="Estado", interactive=False)

        edit_btn.click(fn=ui_edit_text, inputs=[user_choice, operation, input_text, doc_choice], outputs=[edited_out, edit_status])

    with gr.Tab("🧾 Versiones (diff / revert)"):
        v_old = gr.Dropdown(label="Versión OLD", choices=[], value=None)
        v_new = gr.Dropdown(label="Versión NEW", choices=[], value=None)
        diff_btn = gr.Button("Generar diff")
        diff_out = gr.Textbox(label="Diff (unified)", lines=14)

        def _refresh_versions_for_diff(doc_choice):
            opts, _latest = ui_load_versions(doc_choice)
            return opts, opts

        doc_choice.change(fn=_refresh_versions_for_diff, inputs=[doc_choice], outputs=[v_old, v_new])
        diff_btn.click(fn=ui_diff_two_versions, inputs=[doc_choice, v_old, v_new], outputs=[diff_out])

        gr.Markdown("### Revert")
        target_version = gr.Dropdown(label="Versión objetivo", choices=[], value=None)
        doc_choice.change(fn=lambda dc: ui_load_versions(dc)[0], inputs=[doc_choice], outputs=[target_version])

        revert_btn = gr.Button("Revertir a versión")
        revert_msg = gr.Textbox(label="Resultado", interactive=False)
        revert_btn.click(fn=ui_revert, inputs=[user_choice, doc_choice, target_version], outputs=[revert_msg])

    with gr.Tab("💬 Comentarios & Aprobación"):
        comment_text = gr.Textbox(label="Comentario", lines=3)
        add_comment_btn = gr.Button("Añadir comentario")
        comment_status = gr.Textbox(label="Estado", interactive=False)
        add_comment_btn.click(fn=ui_add_comment, inputs=[user_choice, doc_choice, versions_dd, comment_text], outputs=[comment_status])

        list_comments_btn = gr.Button("Listar comentarios")
        comments_json = gr.JSON(label="Comentarios")
        list_comments_btn.click(fn=ui_list_comments, inputs=[doc_choice], outputs=[comments_json])

        gr.Markdown("### Aprobación (approver/admin)")
        appr_status = gr.Radio(label="Estado", choices=["pending","approved","rejected"], value="pending")
        appr_note = gr.Textbox(label="Nota", lines=2)
        approve_btn = gr.Button("Registrar aprobación")
        approve_msg = gr.Textbox(label="Resultado", interactive=False)
        approve_btn.click(fn=ui_approve, inputs=[user_choice, doc_choice, versions_dd, appr_status, appr_note], outputs=[approve_msg])

        list_appr_btn = gr.Button("Ver aprobaciones")
        appr_json = gr.JSON(label="Aprobaciones")
        list_appr_btn.click(fn=ui_list_approvals, inputs=[doc_choice], outputs=[appr_json])

    with gr.Tab("⚙️ Admin (usuarios)"):
        gr.Markdown("Solo rol **admin** puede crear usuarios en esta demo.")
        new_username = gr.Textbox(label="Nuevo username")
        new_role = gr.Dropdown(label="Rol", choices=list(ROLE_PERMISSIONS.keys()), value="designer")
        add_user_btn = gr.Button("Crear usuario")
        add_user_msg = gr.Textbox(label="Resultado", interactive=False)
        add_user_btn.click(fn=ui_add_user, inputs=[user_choice, new_username, new_role], outputs=[add_user_msg])

        refresh_users_btn = gr.Button("Refrescar dropdown de usuarios (login)")
        refresh_users_btn.click(fn=format_user_choices, inputs=None, outputs=user_choice)

demo.queue().launch(share=False)


/tmp/ipython-input-3740303147.py:153: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="Unidad 3 · Bedrock GenAI App", theme=gr.themes.Soft()) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## 13) Pautas éticas y seguridad (checklist)
- **Privacidad**: evita introducir datos personales sensibles en prompts. En producción: cifrado con KMS, control de acceso (IAM), logs mínimos, retención.
- **Derechos de autor**: prohibir prompts que pidan “copiar” marcas/obras protegidas; registrar procedencia y uso.
- **Sesgos**: incorporar revisión humana para contenidos de alto impacto; ajustar políticas y guardrails.
- **Moderación**: bloquear contenido sexual explícito, violencia, odio, etc. (Guardrails recomendado).
- **Auditoría**: trazabilidad por usuario/rol y registro de operaciones (en producción: CloudWatch + CloudTrail).

> Nota: En esta demo, la moderación es **básica** si no se configuran Guardrails. Para un entorno real, usa Bedrock Guardrails y políticas formales.
